<a href="https://colab.research.google.com/github/OlegBEZb/fact-ai/blob/main/SGNS_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U gensim

word level https://www.salesforce.com/products/einstein/ai-research/the-wikitext-dependency-language-modeling-dataset/

In [1]:
!unzip drive/MyDrive/fact-ai/wikitext-103-v1.zip -d .

Archive:  drive/MyDrive/fact-ai/wikitext-103-v1.zip
   creating: ./wikitext-103/
  inflating: ./wikitext-103/wiki.test.tokens  
  inflating: ./wikitext-103/wiki.valid.tokens  
  inflating: ./wikitext-103/wiki.train.tokens  


# Import libs

In [1]:
from IPython.display import clear_output

import hashlib
import re
import sys
import tarfile
from collections import Counter, defaultdict
from pathlib import Path

import matplotlib.pyplot as plt
import requests
from IPython.display import Image

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('all')

clear_output() 

# Read the Data

In [22]:
from pathlib import Path
# Read train, val, and test sets into string objects
train_data = Path('wikitext-103/wiki.train.tokens').read_text()
val_data = Path('wikitext-103/wiki.valid.tokens').read_text()
test_data = Path('wikitext-103/wiki.test.tokens').read_text()

In [25]:
sentences = train_data.split('.')

In [3]:
%%time

import re

# Remove casing, punctuation, special characters, and stop words and also lemmatize the words on a subset of the first 110 articles in the train data
# my_new_text = re.sub('[^ a-zA-Z0-9]|unk', '', train_data[:10000000])
sentences = [re.sub('[^ a-zA-Z0-9]|unk', '', s) for s in sentences]

sentences = [word_tokenize(d.lower()) for s in sentences]

stop_words = set(stopwords.words('english'))
filtered_sentence = (w for s in sentences for w in s if w not in stop_words)

lemma = WordNetLemmatizer()
normalized = " ".join(lemma.lemmatize(word) for word in filtered_sentence)

CPU times: user 12.1 s, sys: 229 ms, total: 12.3 s
Wall time: 12.3 s


In [23]:
train_data[:1000]

' \n = Valkyria Chronicles III = \n \n Senjō no Valkyria 3 : <unk> Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " <unk> Raven " . \n The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more forgiving for series new

In [21]:
my_new_text[:1000]

'   Valkyria Chronicles III    Senj no Valkyria 3   Chronicles  Japanese  3  lit  Valkyria of the Battlefield 3   commonly referred to as Valkyria Chronicles III outside Japan  is a tactical role  playing video game developed by Sega and MediaVision for the PlayStation Portable  Released in January 2011 in Japan  it is the third game in the Valkyria series  Employing the same fusion of tactical and real  time gameplay as its predecessors  the story runs parallel to the first game and follows the  Nameless   a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit   Raven    The game began development in 2010  carrying over a large portion of the work done on Valkyria Chronicles II  While it retained the standard features of the series  it also underwent multiple adjustments  such as making the game more forgiving for series newcomers  Character designer  Honjou and composer Hitoshi Saki

In [4]:
from gensim.models import Word2Vec
import multiprocessing

In [5]:
model = Word2Vec(
    # sentences=word_tokens,  # a list of lists of tokens
                 sg=1,
                 negative=5,
                 vector_size=100, 
                 window=5, 
                 min_count=10, 
                 workers=multiprocessing.cpu_count())

In [6]:
from time import time

t = time()

model.build_vocab(word_tokens, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

Time to build vocab: 0.02 mins


In [7]:
t = time()

model.train(word_tokens, 
            total_examples=model.corpus_count, 
            epochs=30, 
            report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

Time to train the model: 2.04 mins


In [8]:
model.save("word2vec.model")

In [9]:
model.init_sims(replace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  """Entry point for launching an IPython kernel.


In [14]:
model.wv.most_similar(positive=["fat"])

KeyError: ignored

In [ ]:
model.wv.similarity("moe_'s", 'tavern')